Basic Random Forest multiclass regression with default (3, 3) parameter FV on only the MFCC features.

In [8]:
import os
import scipy.io as spio
import numpy as np
from os.path import join, isdir, isfile
%matplotlib inline
import matplotlib.pyplot as plt

DATA_DIR = '../data'
# 'file_name' 'class_name' 'class' 'eng' 'mfc' 'chroma' 't'
# 'keystrength' 'brightness' 'zerocross' 'roughness' 'inharmonic' 'hcdf'
#
FVs = join(DATA_DIR, 'FV.mat')
LBs = join(DATA_DIR, 'LB.mat')
if not isfile(FVs) or not isfile(LBs):
    print('Generating Fisher Vectors')
    !matlab -nodisplay -nosplash -nodesktop -r "run('../tools/FV_concat.m');exit;"
else:
    print('Using existing FVs')

mfcc = np.transpose(spio.loadmat(FVs)['FV'])
labels = spio.loadmat(LBs)['LB'][0]
N = mfcc.shape[0]

p = np.random.permutation(N)
mfcc = mfcc[p]
labels = labels[p]

N

Using existing FVs


1000

In [1]:
import os
import scipy.io as spio
import numpy as np
from os.path import join, isdir, isfile
%matplotlib inline
import matplotlib.pyplot as plt


DATA_DIR = '../generated-fv/'
FVs = join(DATA_DIR, 'FV3-3.mat')
LBs = join(DATA_DIR, 'LB.mat')
mfcc = np.transpose(spio.loadmat(FVs)['FV'])
labels = spio.loadmat(LBs)['LB'][0]
N = mfcc.shape[0]

p = np.random.permutation(N)
mfcc = mfcc[p]
labels = labels[p]

N

1000

In [2]:
trN = int(0.8 * N)
teN = N - trN
trX, trY = mfcc[:trN], labels[:trN]
teX, teY = mfcc[-teN:], labels[-teN:]
len(set(trY))

means = np.mean(trX, axis=0)
stddevs = np.std(trX, axis=0)

def summary(x):
    return '[{:.4f}, {:.4f}]'.format(np.ndarray.min(x), np.ndarray.max(x))
print('means in range', summary(means))
print('std in range', summary(stddevs))

# Whitened data
def adjust(x): return (x - means[None, :]) / stddevs[None, :]
trXadj = adjust(trX)
teXadj = adjust(teX)

print('norm. means in range', summary(np.mean(trXadj, axis=0)))
print('norm. std in range', summary(np.std(trXadj, axis=0)))
trX.shape

means in range [-0.0210, 0.0095]
std in range [0.0201, 0.0618]
norm. means in range [-0.0000, 0.0000]
norm. std in range [1.0000, 1.0000]


(800, 960)

In [ ]:
# actual import mfcc from FV
import numpy as np
import random as random
import scipy.stats as stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import zero_one_loss
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import StratifiedKFold

skf = StratifiedKFold(labels, n_folds=8, shuffle=True, random_state=1)
for d in range(1, 11):
    for l in range(1, 11):
        depth = d*5
        learners = l*100
        rf = RandomForestClassifier(n_estimators = learners, max_depth = depth, warm_start = False)
        score = np.average(cross_val_score(rf, mfcc, labels, cv=skf, n_jobs=1))
        print('depth {}, learners {}, score {}'.format(depth, learners, score))

depth 5, learners 100, score 0.578926282051282
depth 5, learners 200, score 0.6033653846153846
depth 5, learners 300, score 0.6049679487179487
depth 5, learners 400, score 0.6030448717948718
depth 5, learners 500, score 0.6000801282051282
depth 5, learners 600, score 0.6052083333333333
depth 5, learners 700, score 0.6060897435897437
depth 5, learners 800, score 0.5959935897435897
depth 5, learners 900, score 0.602323717948718
depth 5, learners 1000, score 0.6050480769230769
depth 10, learners 100, score 0.6204326923076924
depth 10, learners 200, score 0.6403044871794872
depth 10, learners 300, score 0.6435096153846154
depth 10, learners 400, score 0.6350160256410255
depth 10, learners 500, score 0.637099358974359
depth 10, learners 600, score 0.6440705128205129
depth 10, learners 700, score 0.6432692307692307
depth 10, learners 800, score 0.6382211538461539
depth 10, learners 900, score 0.6451121794871795
depth 10, learners 1000, score 0.643349358974359
depth 15, learners 100, score 0.

In [10]:
# actual import mfcc from FV
import numpy as np
import random as random
import scipy.stats as stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import zero_one_loss

for d in range(1, 11):
    for l in range(1, 11):
        depth = d*5
        learners = l*100
        rf = RandomForestClassifier(n_estimators = learners, max_depth = depth, warm_start = False)
        rf.fit(trX,trY) # NOTE DATA ORIENTATION
        print('depth {}, learners {}, train data accuracy {}, test accuracy {}'.format(depth, learners, rf.score(trX, trY), rf.score(teX, teY)))

depth 5, learners 100, train data accuracy 0.8675, test accuracy 0.53
depth 5, learners 200, train data accuracy 0.8775, test accuracy 0.52
depth 5, learners 300, train data accuracy 0.8725, test accuracy 0.525
depth 5, learners 400, train data accuracy 0.87625, test accuracy 0.54
depth 5, learners 500, train data accuracy 0.88, test accuracy 0.535
depth 5, learners 600, train data accuracy 0.88125, test accuracy 0.555
depth 5, learners 700, train data accuracy 0.87375, test accuracy 0.53
depth 5, learners 800, train data accuracy 0.87625, test accuracy 0.53
depth 5, learners 900, train data accuracy 0.88, test accuracy 0.505
depth 5, learners 1000, train data accuracy 0.8775, test accuracy 0.52
depth 10, learners 100, train data accuracy 1.0, test accuracy 0.56
depth 10, learners 200, train data accuracy 1.0, test accuracy 0.59
depth 10, learners 300, train data accuracy 1.0, test accuracy 0.6
depth 10, learners 400, train data accuracy 1.0, test accuracy 0.58
depth 10, learners 500, 